In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
current_folder = globals()['_dh'][0]
os.chdir(os.path.dirname(os.path.abspath(current_folder)))

In [18]:
os.path.dirname(os.path.abspath(current_folder))


'/home/hitesh/challenge/dualmapfree/lib/models/MicKey/modules/ml_fourm'

In [2]:
from PIL import Image
import numpy as np
import torch
from torchvision.transforms.functional import center_crop
from torchvision.utils import make_grid
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
from matplotlib import gridspec

try:
    import faiss
except:
    print('Please install faiss via pip install faiss-gpu to perform retrieval.')
    
from fourm.models.fm import FM
from fourm.vq.vqvae import VQVAE, DiVAE
from fourm.models.generate import GenerationSampler, build_chained_generation_schedules, init_empty_target_modality, init_full_input_modality, custom_text
from fourm.data.modality_transforms import RGBTransform
from fourm.data.modality_info import MODALITY_INFO
from fourm.data.modality_transforms import MetadataTransform
from fourm.utils.plotting_utils import decode_dict, visualize_bboxes, plot_text_in_square, decode_tok_depth, decode_tok_semseg

Please install faiss via pip install faiss-gpu to perform retrieval.


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations
INFO:datasets:PyTorch version 2.0.1 available.


No module named 'detectron2'
Detectron2 can be used for semseg visualizations. Please install detectron2 to use this feature, or plotting will fall back to matplotlib.
No module named 'pyrender'
Human pose dependencies are not installed, hence poses will not be visualized. To visualize them (optional), you can do the following: 
1) Install via `pip install timm yacs smplx pyrender pyopengl==3.1.4` 
   You may need to follow the pyrender install instructions: https://pyrender.readthedocs.io/en/latest/install/index.html 
2) Download SMPL data from https://smpl.is.tue.mpg.de/. See https://github.com/shubham-goel/4D-Humans/ for an example. 
3) Copy the required SMPL files (smpl_mean_params.npz, SMPL_to_J19.pkl, smpl/SMPL_NEUTRAL.pkl) to fourm/utils/hmr2_utils/data .


In [3]:
# The flag below controls whether to allow TF32 on matmul. This flag defaults to False in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True
# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_grad_enabled(False)

In [4]:
text_tok = Tokenizer.from_file('./fourm/utils/tokenizer/trained/text_tokenizer_4m_wordpiece_30k.json')

toks = {
    'tok_dinov2': VQVAE.from_pretrained('EPFL-VILAB/4M_tokenizers_DINOv2-B14_8k_224-448').eval().to(device),
}

In [5]:
fm = FM.from_pretrained('EPFL-VILAB/4M-21_XL').eval().to(device)
sampler = GenerationSampler(fm)

In [6]:
cond_domains = ['rgb@224']
target_domains = ['tok_dinov2@224'] 
tokens_per_target = [256]
autoregression_schemes = ['roar']
decoding_steps = [1]
token_decoding_schedules = ['linear']
temps = [0.01]
temp_schedules = ['constant'] 
cfg_scales = [2.0]
cfg_schedules = ['constant'] 
cfg_grow_conditioning = True
top_p, top_k = 0.8, 0.0

schedule = build_chained_generation_schedules(
    cond_domains=cond_domains, target_domains=target_domains, tokens_per_target=tokens_per_target, autoregression_schemes=autoregression_schemes, 
    decoding_steps=decoding_steps, token_decoding_schedules=token_decoding_schedules, temps=temps, temp_schedules=temp_schedules,
    cfg_scales=cfg_scales, cfg_schedules=cfg_schedules, cfg_grow_conditioning=cfg_grow_conditioning, 
)

In [7]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformations
rgb_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def preprocess_image(image_tensor):
    images = []
    for img in image_tensor:
        img_pil = transforms.ToPILImage()(img)
        img_pil = rgb_transform(img_pil)
        images.append(img_pil)
    return torch.stack(images)

img = torch.load("/home/hitesh/challenge/dualmapfree/te")

img_process = preprocess_image(img)


In [8]:
batched_sample = {
    'rgb@224': {
        'tensor': img_process.cuda(), # Batched tensor
        'input_mask': torch.zeros(12, 196, dtype=torch.bool, device=device), # False = used as input, True = ignored
        'target_mask': torch.ones(12, 196, dtype=torch.bool, device=device), # False = predicted as target, True = ignored
    }
}

# Initialize target modalities
for target_mod, ntoks in zip(target_domains, tokens_per_target):
    batched_sample = init_empty_target_modality(batched_sample, MODALITY_INFO, target_mod, 12, ntoks, device)
    
# Initialize input modalities
for cond_mod in cond_domains:
    batched_sample = init_full_input_modality(batched_sample, MODALITY_INFO, cond_mod, device, eos_id=text_tok.token_to_id("[EOS]"))

In [9]:
# batched_sample

In [10]:
out_dict = sampler.generate(
    batched_sample, schedule, text_tokenizer=text_tok, 
    verbose=True, seed=0,
    top_p=top_p, top_k=top_k,
)
dec_dict = decode_dict(
    out_dict, toks, text_tok, 
    image_size=224, patch_size=16,
    decoding_steps=50
)

1it [00:00,  1.72it/s]


In [12]:
dec_dict['tok_dinov2@224'].shape

torch.Size([12, 768, 16, 16])